In [ ]:
# Colab: run this once
!pip install -q langchain langchain-openai langchain-community langchain-text-splitters \
               chromadb sentence-transformers pypdf gradio openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.5 MB/s eta 0:

In [ ]:
from google.colab import userdata
import os

# Fetch your stored Colab secrets
NEBIUS_KEY = userdata.get("NEBIUS_API_KEY")
OPENAI_KEY = userdata.get("OPENAI_API_KEY")

# Check what loaded (safe because values are hidden)
print("NEBIUS_KEY found:", bool(NEBIUS_KEY))
print("OPENAI_KEY found:", bool(OPENAI_KEY))
NEBIUS_BASE = "https://api.studio.nebius.com/v1"



NEBIUS_KEY found: True
OPENAI_KEY found: True


In [ ]:
# LLM (generation)
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    base_url=NEBIUS_BASE,
    api_key=NEBIUS_KEY,
    temperature=0.0,
)

# Loaders & chunking
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Embeddings (local, free)
from langchain.embeddings import HuggingFaceEmbeddings
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Vector store
from langchain_community.vectorstores import Chroma


/tmp/ipython-input-2894255733.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def build_kb(pdf_paths, chunk_size=800, chunk_overlap=120, k=3):
    """Return a LangChain retriever from one or more PDF file paths."""
    if not pdf_paths:
        raise ValueError("Pass at least one PDF path")

    # Load pages from all PDFs
    docs = []
    for p in pdf_paths:
        docs.extend(PyPDFLoader(p).load())  # each Document has .page_content and metadata (page)

    # Chunk
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_documents(docs)
    chunks = [c for c in chunks if c.page_content and len(c.page_content.strip()) > 10]

    # Build vector DB in-memory and return retriever
    vectordb = Chroma.from_documents(chunks, embedding=emb, collection_name="pdf_kb")
    retriever = vectordb.as_retriever(search_kwargs={"k": k})
    return retriever


In [ ]:
Overlap = 10–20% of chunk size

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

SYSTEM = "You are a helpful assistant. Answer ONLY using the provided context. If the answer is not in the context, say you don't know."

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM),
    ("human", "Question: {question}\n\nContext:\n{context}")
])

def to_context_with_sources(docs):
    # nice formatted context and source list
    ctx = "\n\n".join(d.page_content for d in docs)
    srcs = []
    for i, d in enumerate(docs):
        pg = d.metadata.get("page", "?")
        preview = d.page_content.strip().replace("\n", " ")
        if len(preview) > 220: preview = preview[:220] + "..."
        srcs.append(f"[{i+1}] p.{pg} — {preview}")
    return {"context": ctx, "sources_text": "\n".join(srcs)}

def make_rag_chain(retriever):
    # map: question -> {context, sources_text} -> prompt -> llm -> text
    def _retrieve(q):
        docs = retriever.get_relevant_documents(q)
        out = to_context_with_sources(docs)
        out["question"] = q
        return out

    chain = (
        RunnableLambda(_retrieve)
        | (lambda d: prompt.format_messages(question=d["question"], context=d["context"]))
        | llm
        | StrOutputParser()
    )
    return chain


In [ ]:

from google.colab import files
uploaded = files.upload()  # pick 1–N PDFs
pdf_paths = [f"/content/{name}" for name in uploaded.keys()]

retriever = build_kb(pdf_paths, chunk_size=800, chunk_overlap=120, k=3)
rag_chain = make_rag_chain(retriever)

def ask(q: str, show_sources=True):
    answer = rag_chain.invoke(q)
    print("Q:", q)
    print("\nAnswer:\n", answer)
    if show_sources:
        # get sources by re-running retrieve for display
        docs = retriever.get_relevant_documents(q)
        srcs = []
        for i, d in enumerate(docs):
            pg = d.metadata.get("page", "?")
            preview = d.page_content.strip().replace("\n", " ")
            if len(preview) > 220: preview = preview[:220] + "..."
            srcs.append(f"[{i+1}] p.{pg} — {preview}")
        print("\n—\nSources:\n" + "\n".join(srcs))


Saving pdftrainagle.pdf to pdftrainagle.pdf


In [ ]:
ask("how many different categories have they tested on?")

/tmp/ipython-input-2113940531.py:26: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(q)


Q: how many different categories have they tested on?

Answer:
 10 different categories.

—
Sources:
[1] p.4 — from various domains, then subsampled 100 docu- ments based on their reading level (Flesch, 1948).4 By collecting a broad set of document-oriented questions, we built a robust set of tasks across in- dustries for testing...
[2] p.6 — 2. The category of the question, using the taxon- omy in section 4. 3. The ranking of each generated answer for the given question-document pair. 4. The accuracy, informativeness, readabil- ity/understandability, and cla...
[3] p.16 — Figure 15: Informativeness Annotation Scores by Ques- tion Category Figure 16: Clarity Annotation Scores by Question Cate- gory Figure 17: Readability Annotation Scores by Question Category


In [ ]:
ask("what are the 10 different categories in this paper?")

Q: what are the 10 different categories in this paper?

Answer:
 The 10 different categories in this paper are:

1. Figure Questions
2. Text Questions
3. Table Reasoning
4. Structure Questions
5. Summarization
6. Extraction

—
Sources:
[1] p.11 — Classification Is this document a scientific article? Is this document about a residential lease or a commercial lease? Trick Question A good trick question might: (a) be related to the document (b) refer to non-existent...
[2] p.11 — Category Positive Examples Figure Questions What is the main takeaway of Figure 4? What is the largest value in Figure 4? What kind of graph is used on page 5? Text Questions Is 2pm on Wednesday free? What evidence is us...
[3] p.16 — Figure 15: Informativeness Annotation Scores by Ques- tion Category Figure 16: Clarity Annotation Scores by Question Cate- gory Figure 17: Readability Annotation Scores by Question Category
